# 有关pytorch自动求导机制的探索与展示

pytorch的自动求导机制是一个强大的功能，本人（笨人）在此尝试边学边写一个文件用来展示，有不正确的地方欢迎骂我菜就多练并联系我
[指正](mailto:12410615@mail.sustech.edu.cn)

## 叶子节点与非叶子节点

pytorch的操作围绕张量展开，而pytorch的张量又可以分为两种，一种是叶子张量，一种是非叶子张量。
叶子张量，简单理解，就是某个枝干的尽头，它代表某个“end”，叶子会连在枝干上，但不会有东西连在叶子上。在实际操作中，叶子张量就是指用户直接创建的张量，而不是通过某些函数操作生成的张量。

非叶子张量，与叶子张量相对，它是有迹可循的，是被叶子连接的枝干，一个不太恰当的解释是，非叶子张量是叶子张量通过一些函数操作得到的中间张量（这种解释真的及其不恰当，但可以先这么简单理解）

In [ ]:
# 代码展示
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# 框框引入一堆包

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# 电脑上没有nvidia显卡，（默默流下眼泪），就用cpu来展示叭

cpu


In [27]:
a = torch.tensor([1.0], requires_grad=True) # 这里创建一个tensor，它是一个源头，是我们直接手动创建的，应该是一个叶子节点
print(a.is_leaf) # 检验一下，发现确实是一个叶子节点

b = a * 2 # 某种函数操作
print(b.is_leaf) # 发现b不是叶子节点，因为它是由a计算出来的


True
False


## grad与grad_fn
### grad
为什么需要区分叶子张量与非叶子张量？我们可以思考一下我们的优化方法（梯度下降那一坨），我们的任务是优化我们的权重，调整权重的值来给出更好的输出。此处我们的权重就是我们首先初始化（直接创建，是叶子张量），然后用梯度调整的值，它需要grad(梯度)。而中间生成的张量，并不是我们需要在意的东西，简言之，我们只需要把梯度从非叶子张量中间传回去，而不需要保留非叶子张量的梯度，因为我们的终极任务是调整权重大小（叶子张量数值），非叶子张量不是我们的目标。

In [ ]:
# 我们刚刚创建了a，b张量，这里拿来展示
b.backward() # 反向传播，计算梯度
print(a.grad) # 打印一下a的梯度，发现确实是2.0 (db/da = 2a = 2*1.0 = 2.0)
print(b.grad) # 打印一下b的梯度，发现b的梯度是None，因为b不是叶子节点，我们并不在意，所以没有梯度


tensor([2.])
None


C:\Users\w1369\AppData\Local\Temp\ipykernel_22888\3387406872.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\build\aten\src\ATen/core/TensorBody.h:494.)
  print(b.grad) # 打印一下b的梯度，发现b的梯度是None，因为b不是叶子节点，我们并不在意，所以没有梯度


### grad_fn

一个很有意思的事情，pytorch如何实现这种自动求导机制？我们只需要在张量上进行一个backward方法的调用，就可以算出之前的梯度值。

答案是，pytorch会保存中间张量（非叶子张量）的生成操作，我们刚才的非叶子张量b是通过a*2得到的，所以pytorch会自动记录这种操作，在backward反向传播的时候，会读取这种操作的反向传播方式，然后把梯度回传回去。

In [ ]:
print(a.grad_fn) # 打印一下a的grad_fn，发现是None，因为a是一个叶子节点
print(b.grad_fn) # 打印一下b的grad_fn，发现是一个MulBackward对象，说明b是由a计算出来的，这里MulBackward代表b是a乘出来的
# backward反向传播时，为了把b的梯度传递给a，PyTorch会读取这种操作的梯度反向传递方式

None


但是如果为了一些其他的操作，这里我们也可以保留b的梯度

In [28]:
b.retain_grad() # 让b保留梯度
b.backward() # 反向传播，计算梯度
print(a.grad)
print(b.grad)
# 芜湖，我们得到了和之前不一样的结果，db/db = 1.0，b的梯度在这里也成功的保留在了grad里边，但是注意，b仍然是非叶子节点。
print(b.is_leaf) # 仍然是非叶子节点

tensor([2.])
tensor([1.])
False


## detach
一个很有意思的操作是detach操作，它的操作原理是"创建一个新的 Tensor，从当前计算图中分离出来，新的 Tensor 不具备梯度信息。"
(截取自grok)
我们可以进一步的思考这会带来什么效果，这个张量会被创建出来，然后它理应是一个新的叶子节点，而此处由于grad_fn会指向None，我们
将无法把梯度回传给上游，下边我们来验证一下我们的猜想

In [38]:
a = torch.tensor([1.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)
c = torch.tensor([1.0], requires_grad=True) 
# 一口气创建三个叶子张量
d = a + b + c
# 这里创建了一个新的张量d，它是由a、b、c计算出来的
e = d
# 我们把d的引用传给e，现在e是由一个等式操作得到的非叶子张量，我们验证一下我们的猜想
print(e.is_leaf) # False,发现e不是叶子节点
print(e.grad_fn) # 存储了生成e的function操作信息

# 接下来我们给它上压力，detach一下
e = e.detach()
# 现在我们把e给截断，从原有计算图中分离出来
print(e.is_leaf) # True,发现e是叶子节点
print(e.grad_fn) # 发现e的grad_fn是None，因为它是一个叶子节点
e.requires_grad_() # 让e重新需要梯度
h = e * 2
# 既然e已经从原有的计算图中截取出来，那么此处如果从h调用backward方法，那么理论上来讲梯度信息不会回传给abc，
# 只能回传给e，我们验证一下猜想
h.backward()
print(a.grad) # None,发现a的梯度是None
print(e.grad) # 发现e的梯度是1.0，说明h的梯度确实回传给了e

False
True
None
None
tensor([2.])


## requires_grad?
刚刚其实我一直有意的回避requires_grad这个属性的影响，因为我们可以想想，对于模型的训练阶段，我们会需要保留梯度信息，以便进行梯度下降调整权重，但是在模型的推理阶段，这种梯度就是多余的操作，我们并不需要它。也就是说，总会存在不同的场景，我们的叶子张量需要梯度或不需要。

In [ ]:
a = torch.tensor([1.0], requires_grad=False) # 默认情况下requires_grad其实也是False
b = torch.tensor([1.0], requires_grad=False)
c = a + b
print(c.requires_grad) # False,发现c的requires_grad也是False
# 生成c的两个张量不需要梯度，所以c也不需要梯度
# 那如果b需要梯度呢？
b.requires_grad_() # 让b需要梯度
c = a + b
print(c.requires_grad) # False,发现c的requires_grad变为了True
# 这说明了一个问题，c的requires_grad是由它的输入决定的，而不是由它的输出决定的
c.backward() # 反向传播，计算梯度
print(a.grad) # None,发现a的梯度是None,(因为a的requires_grad是False)
print(b.grad) # 发现b的梯度是1.0，说明c的梯度确实回传给了b

False
True
None
tensor([1.])


嗯，就写到这了，我太懒了。
我是菜狗，欢迎说我菜就多练并指正！